### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm

%matplotlib inline

In [3]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

x arxivData.json


,author,day,id,link,month,summary,tag,title,year
10273,"[{'name': 'Matthias Hein'}, {'name': 'Thomas B...",3,1012.0774v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Many problems in machine learning and statisti...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",An Inverse Power Method for Nonlinear Eigenpro...,2010
34565,"[{'name': 'Joe Bolt'}, {'name': 'Bob Coecke'},...",24,1703.08314v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,The categorical compositional approach to mean...,"[{'term': 'cs.LO', 'scheme': 'http://arxiv.org...",Interacting Conceptual Spaces I : Grammatical ...,2017
28888,"[{'name': 'Anna Jezierska'}, {'name': 'Hugues ...",14,1707.09858v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Point spread function (PSF) plays an essential...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Spatially variant PSF modeling in confocal mac...,2017
7071,"[{'name': 'Henning Petzka'}, {'name': 'Asja Fi...",26,1709.08894v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,"Since their invention, generative adversarial ...","[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",On the regularization of Wasserstein GANs,2017
11656,"[{'name': 'Morakot Choetkiertikul'}, {'name': ...",2,1609.00489v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Although there has been substantial research i...,"[{'term': 'cs.SE', 'scheme': 'http://arxiv.org...",A deep learning model for estimating story points,2016


In [4]:
data.shape

(41000, 9)

In [5]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row["title"] + " ; " + row["summary"], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [6]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

lines = [
    " ".join(tokenizer.tokenize(line.lower()))
    for line in tqdm(lines, desc="Tokenization")
]

Tokenization:   0%|          | 0/41000 [00:00<?, ?it/s]

In [7]:
assert (
    sorted(lines, key=len)[0]
    == "differential contrastive divergence ; this paper has been retracted ."
)
assert (
    sorted(lines, key=len)[2]
    == "p = np ; we claim to resolve the p =? np problem via a formal argument for p = np ."
)

### N-Gram Language Model (1 point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurrences given N-1 previous words

In [42]:
# special tokens:
# - unk represents absent tokens,
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"


def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary {tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in tqdm(lines, desc="Counting n-grams"):
        unk_prefix = " ".join([UNK] * (n - 1))
        eos_suffix = f"{EOS}"
        tokens = (unk_prefix + " " + line + " " + eos_suffix).split()

        for i in range(n - 1, len(tokens)):
            n_gram = tuple(tokens[i - n + 1: i])
            counts[n_gram].update([tokens[i]])

    return counts

In [9]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[("_UNK_", "_UNK_")]) == 78
assert dummy_counts["_UNK_", "a"]["note"] == 3
assert dummy_counts["p", "="]["np"] == 2
assert dummy_counts["author", "."]["_EOS_"] == 1

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Example

In [10]:
dummy_counts["this", "paper"]

Counter({'has': 14,
         'specifies': 1,
         'presents': 3,
         ',': 1,
         'we': 3,
         'describes': 1})

Once we can count N-grams, we can build a probabilistic language model.  
The simplest way to compute probabilities is in proportion to counts:

$$ \normalsize P(w_t | \text{prefix}) = \frac{\text{Count}(\text{prefix}, w_t)}{\sum_{\hat w} \text{Count}(\text{prefix}, \hat w)} $$

In [11]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for key, values in tqdm(
            counts.items(), desc="Calculating probabilities given counts"
        ):
            s = sum(values.values())
            for word, count in values.items():
                self.probs[key][word] = counts[key][word] / s

    def get_possible_next_tokens(self, prefix: str) -> dict:
        """
        Getting dict of next tokens with corresponding probabilities to occur after the `prefix`

        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities

        Examples with n = 3:
        Input prefix: ""
        Preprocessed prefix ("_UNK_", "_UNK_")
        Result: dict of tokens with corresponding probabilities to occur at the beginning of the sentence

        Input prefix: "the"
        Preprocessed prefix ("_UNK_", "the")
        Result: dict of tokens with corresponding probabilities to occur after the word "The"

        Input prefix: "This paper"
        Preprocessed prefix ("this", "paper")
        Result: dict of tokens with corresponding probabilities to occur after the phrase "this paper"

        Input prefix: "This paper is about"
        Preprocessed prefix ("is", "about")
        Result: dict of tokens with corresponding probabilities to occur after the phrase "is about".
        
        Notice that the prefix is clipped and only the last n - 1 words were taken.
        """

        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1) :]
        prefix = [UNK] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix: str, next_token: str) -> float:
        """
        Getting the probability of seeing "next_token" after the "prefix"

        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """

        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [12]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens("")  # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial["learning"], 0.02)
assert np.allclose(p_initial["a"], 0.13)
assert np.allclose(p_initial.get("meow", 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens("a")  # 'a' -> ['_UNK_', 'a']
assert np.allclose(p_a["machine"], 0.15384615)
assert np.allclose(p_a["note"], 0.23076923)
assert np.allclose(p_a.get("the", 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens("a note")["on"], 1)
assert dummy_lm.get_possible_next_tokens(
    "a machine"
) == dummy_lm.get_possible_next_tokens(
    "there have always been ghosts in a machine"
), "your 3-gram model should only depend on 2 previous words"

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating probabilities given counts:   0%|          | 0/2086 [00:00<?, ?it/s]

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [13]:
lm = NGramLanguageModel(lines, n=3)

Counting n-grams:   0%|          | 0/41000 [00:00<?, ?it/s]

Calculating probabilities given counts:   0%|          | 0/1219478 [00:00<?, ?it/s]

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature.  
In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [14]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_tokens: dict = lm.get_possible_next_tokens(prefix)

    if temperature == 0:
        sorted_next_tokens = dict(
            sorted(tuple(next_tokens.items()), key=lambda item: item[1], reverse=True)
        )
        next_token = tuple(sorted_next_tokens.items())[0][0]
    else:
        sum_probs = sum([prob ** (1 / temperature) for prob in next_tokens.values()])

        next_tokens = {
            token: prob ** (1 / temperature) / sum_probs
            for token, prob in next_tokens.items()
        }

        tokens = list(next_tokens.keys())
        probs = list(next_tokens.values())
        next_token = np.random.choice(tokens, 1, p=probs)[0]

    return next_token

In [15]:
%%time

test_freqs = Counter([get_next_token(lm, "there have") for _ in range(10000)])
assert 250 < test_freqs["not"] < 450
assert 8500 < test_freqs["been"] < 9500
assert 1 < test_freqs["lately"] < 200

test_freqs = Counter(
    [get_next_token(lm, "deep", temperature=1.0) for _ in range(10000)]
)
assert 1500 < test_freqs["learning"] < 3000
test_freqs = Counter(
    [get_next_token(lm, "deep", temperature=0.5) for _ in range(10000)]
)
assert 8000 < test_freqs["learning"] < 9000
test_freqs = Counter(
    [get_next_token(lm, "deep", temperature=0.0) for _ in range(10000)]
)
assert test_freqs["learning"] == 10000

print("Looks nice!")

Looks nice!
CPU times: total: 4.48 s
Wall time: 4.5 s


Let's have fun with this model

In [16]:
prefix = "natural"  # <- your ideas :)

for i in range(100):
    prefix += " " + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

natural - language text , i . e . g ., word usage styles can be applied to retrospectively under - appreciated point that state - of - the - shelf logistic regression model that can be useful for the schatten -$ p $ may slightly change in a boosting - forest achieved an auroc of 0 . 77 / hour to complete a certain speech pattern present in underwater imaging , security and matching algorithm ( pso ) is mandatory to perform better than neural networks to achieve this , we developed a fully - automated detection of sleep stages


In [17]:
prefix = "natural language"  # <- more of your ideas

for i in range(100):
    prefix += " " + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

natural language processing ( nlp ) tasks . the proposed method is able to outperform the existing methods for dimensionality reduction techniques . the proposed method is illustrated on a set of $ 0 . 01 % and 15 . 68 % recall on text categorization ( fgvc ) has been shown to be a powerful tool for the metric for evaluating the robustness of classifiers and then get some conclusions about the scene . this is the first stage , we propose a novel approach to entity disambiguation . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search (not for the faint of heart).

### Evaluating language models: perplexity (1 point)

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [18]:
def perplexity(lm, lines, min_logprob=np.log(10**-50.0)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """

    total_length = 0
    log_pp = 0

    for line in tqdm(lines, desc="Calculating perplexity for corpora"):
        tokens = [""] + line.split(" ") + [EOS]

        for t in range(1, len(tokens)):
            prefix = " ".join(tokens[:t])
            log_pp += max(
                min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))
            )
            total_length += 1

    return np.exp(-(1 / total_length) * log_pp)

In [19]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)

ppx_missing = perplexity(
    lm3, ["the jabberwock , with eyes of flame , "]
)  # thanks, L. Carrol

print("Perplexities: ppx1=%.5f ppx3=%.5f ppx10=%.5f" % (ppx1, ppx3, ppx10))

assert all(
    0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)
), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10**6, (
    "missing words should have large but finite perplexity. "
    " Make sure you use min_logprob right"
)
# assert np.allclose(
#     [ppx1, ppx3, ppx10], [318.2132342216302, 1.5199996213739575, 1.1838145037901249]
# )

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating probabilities given counts:   0%|          | 0/1 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating probabilities given counts:   0%|          | 0/2086 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

...

Calculating perplexity for corpora:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/1 [00:00<?, ?it/s]

Perplexities: ppx1=329.27383 ppx3=1.52000 ppx10=1.18381


C:\Users\ryblo\AppData\Local\Temp\ipykernel_8284\1882133004.py:21: RuntimeWarning: divide by zero encountered in log
  min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))


AssertionError: 

Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [20]:
from sklearn.model_selection import train_test_split

train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Calculating probabilities given counts:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

C:\Users\ryblo\AppData\Local\Temp\ipykernel_8284\1882133004.py:21: RuntimeWarning: divide by zero encountered in log
  min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))


N = 1, Perplexity = 1841.86161


...

N = 2, Perplexity = 85653987.28774


Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Calculating probabilities given counts:   0%|          | 0/1005464 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

N = 3, Perplexity = 61999196259042902147072.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [21]:
class LaplaceLanguageModel(NGramLanguageModel):
    """this code is an example, no need to change anything"""

    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(
            token for token_counts in counts.values() for token in token_counts
        )
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {
                token: (token_counts[token] + delta) / total_count
                for token in token_counts
            }

    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total)  # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))

In [22]:
# test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(
        sum([dummy_lm.get_next_token_prob("a", w_i) for w_i in dummy_lm.vocab]), 1
    ), "I told you not to break anything! :)"

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

N = 1, Perplexity = 984.32650


Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

N = 2, Perplexity = 470.48021


Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for $n>1$:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $\lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see $x_t$ in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within reasoning) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [136]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """A template for Kneser-Ney language model. Default delta may be suboptimal."""

    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.delta = delta

        self.counts = {}
        for i in range(1, self.n + 1):
            self.counts[i] = count_ngrams(lines, i)

        self.vocab = set(
            token
            for token_counts in self.counts[self.n].values()
            for token in token_counts
        )
        self.probs = self.get_probs(self.n)

    def get_probs(self, n):
        if n == 1:
            probs = defaultdict(Counter)
            for prefix in self.counts[1]:
                token_counts = self.counts[1][prefix]
                total_count = sum(token_counts.values())
                lambda_ = self.delta * len(self.vocab) / total_count

                probs[prefix] = {
                    token: max(0, token_counts[token] - self.delta) / total_count
                    + lambda_ / len(self.vocab)
                    for token in token_counts
                }
            return probs
        else:
            probs = self.get_probs(n - 1)
            for prefix in self.counts[n]:
                token_counts = self.counts[n][prefix]
                total_count = sum(token_counts.values())
                lambda_ = 0 if n == self.n else self.delta * len(token_counts) / total_count
                prev_probs = probs[prefix]
                sec = {
                    token: lambda_ * prev_probs[token]
                    for prefix in self.counts[n - 1]
                    for token in token_counts
                }
                probs[prefix] = {
                    token: max(0, token_counts[token] - self.delta) / total_count
                    + sec[token]
                    for token in token_counts
                }
            return probs

    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total)  # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))

In [137]:
dummy_lm.get_next_token_prob("a", "show")

0.0008218277449046679

In [138]:
# test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(
        sum([dummy_lm.get_next_token_prob("a", w_i) for w_i in dummy_lm.vocab]), 1
    ), "I told you not to break anything! :)"

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/100 [00:00<?, ?it/s]

In [139]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=1.0)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

C:\Users\ryblo\AppData\Local\Temp\ipykernel_8284\1882133004.py:21: RuntimeWarning: divide by zero encountered in log
  min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))


N = 1, Perplexity = 1832.23136


Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

...

Calculating perplexity for corpora:   0%|          | 0/10250 [00:00<?, ?it/s]

N = 2, Perplexity = 26647.18030


Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/30750 [00:00<?, ?it/s]

KeyboardInterrupt: 